In [1]:
%load_ext autoreload
%autoreload 2
%cd ./src/tool/

/home/ec2-user/SageMaker/merck_nl2sql/src/tool


# Merck Graphical User Interface (GUI)

<font color='red'>Disclaimer: This is a pilot of the real tool.</font>

Notebook for Merck users to interact with the Natural Language to SQL tool.


## Start the GUI 

The GUI will appear after running the cell below. To do so you can select the cell and click `▶︎⎮ Run`.

## Overview

    1. Set the credentials.
    2. Introduce the natural language query in the main tab. Hit "detect".
    3. Review that the query appearing in the box below is correct and entities are mapped to the correct ICD10 codes.
    4. (optional) If any corrections are required use the "correct detection" or "correct code map" tabs as needed.
    5. Hit "Execute" to see the requested table.


## In-depth

### Credentials interface

Merck require users to have access to the data in Redshift. To ensure the security of Merck data users are required to provide their individual credentials to use the tool. The credentials are used to "Execute" the queries against the DB. 


### Main interface

The interface is composed of three tabs: `Main`, `Correct detection` and `Correct code map`. 

* The `main` tab is used to input the Natural Language Query and has two buttons: 

    1. `Detect` button runs key entities detection and maps key entities to the most likely option.
    2. `Execute` is intended to be runned after the user verifies that the detection and disambiguation are correct. It will run the query agains the Data Base and return the desired table. 
    
* The `Correct detection` is intended for the user to add terms in the sentence that hasen't been automatically detected.

    1. The `Write name` box is intended for the user to write the term that hasn't been detected.
    2. The `Category` box is intended for the user to choose the category of the entity highlighted by the user.
    3. The `Highlight` button is intended for the user to submit the higlighted term. The display will be automatically updated when the highlight is submitted. 
    
* The `Correct code map` is intended for the user to rectify mappings of sentence terms to RxNorm and ICD10 codes for drugs and conditions respectively.

    1. The `Drug (Condition)` box is inteded for users to select the detected Drug (Condition) for which they want to change the mapping.
    2. The `Show drug (condition) info` will display iformation introduced in the Drug (Condition) including mapping recommendations and the RxNorm (ICD10) code it is currently mapping to.
    3. The `Map to` box is intended for the user to write the RxNorm (ICD10) code they want the drug (condition) to map to.
    4. The `Update drug (condition)` button is intended for the user to submit the mapping update.
    

----

**Example question**

Questions where arguments will be detected:

    * Number of patients taking Aspirin
    * Number of patients having Insomnia
    
Questions where arguments will not be detected:
    
    * Number of patients by gender
    * Number of patients by race

----

In [2]:
import warnings; warnings.filterwarnings('ignore')
import config
from engine.pipeline import nlq2SqlTool
from ui.ui import UI

tool = nlq2SqlTool(config)
ui = UI(tool)
ui.main()

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [3]:
tool.set_db_credentials('master', 'Password123!')

In [4]:
dir(tool)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_close_redshift_connection',
 '_open_redshift_connection',
 '_password',
 '_user',
 'clear_credentials',
 'config',
 'credentials_exist',
 'detect_entities',
 'execute_sql_query',
 'ml_call',
 'model',
 'process_entities',
 'render_template_query',
 'replace_name_for_placeholder',
 'set_db_credentials']

In [5]:
nlq = 'Number of patients taking Aspirin'
nlq

'Number of patients taking Aspirin'

In [6]:
entities = tool.detect_entities(nlq)
entities

{'TIMEDAYS': [],
 'TIMEYEARS': [],
 'DRUG': [{'BeginOffset': 26, 'EndOffset': 33, 'Text': 'Aspirin'}],
 'CONDITION': [],
 'STATE': [],
 'GENDER': [],
 'ETHNICITY': [],
 'RACE': []}

In [7]:
proc_entities = tool.process_entities(entities)
proc_entities

{'TIMEDAYS': [],
 'TIMEYEARS': [],
 'DRUG': [{'BeginOffset': 26,
   'EndOffset': 33,
   'Text': 'Aspirin',
   'Options': [{'Description': 'aspirin',
     'Code': '1191',
     'Score': 0.941210150718689},
    {'Description': 'Aspirin 325 MG Delayed Release Oral Tablet',
     'Code': '198467',
     'Score': 0.5810876488685608},
    {'Description': 'Aspirin 81 MG Delayed Release Oral Tablet [Aspir-Low]',
     'Code': '1293665',
     'Score': 0.3323255777359009},
    {'Description': 'Aspirin 325 MG Delayed Release Oral Tablet [Ecpirin]',
     'Code': '1001476',
     'Score': 0.32233837246894836},
    {'Description': 'Aspirin 81 MG Delayed Release Oral Tablet [Miniprin]',
     'Code': '1052678',
     'Score': 0.29937997460365295}],
   'Query-arg': '1191',
   'Placeholder': '<ARG-DRUG><0>'}],
 'CONDITION': [],
 'STATE': [],
 'GENDER': [],
 'ETHNICITY': [],
 'RACE': []}

In [8]:
proc_nlq = tool.replace_name_for_placeholder(nlq, proc_entities)
proc_nlq

'Number of patients taking <ARG-DRUG><0>'

In [9]:
general_sql = tool.ml_call(proc_nlq)
general_sql

'SELECT COUNT( DISTINCT pe1.person_id) FROM (<SCHEMA>.person pe1 JOIN (<DRUG-TEMPLATE><ARG-DRUG><0> JOIN <SCHEMA>.drug_exposure dr1 ON concept_id=drug_concept_id) ON pe1.person_id=dr1.person_id);'

In [11]:
executable_sql = tool.render_template_query(general_sql, proc_entities)
executable_sql

"SELECT COUNT( DISTINCT pe1.person_id) FROM (cmsdesynpuf23m.person pe1 JOIN (( SELECT descendant_concept_id AS concept_id FROM (SELECT * FROM (SELECT concept_id_2 FROM ( (SELECT concept_id FROM  cmsdesynpuf23m.concept WHERE vocabulary_id='RxNorm' AND ( concept_code='1191' )) JOIN  ( SELECT concept_id_1, concept_id_2 FROM  cmsdesynpuf23m.concept_relationship WHERE relationship_id='Maps to' )  ON concept_id=concept_id_1) ) JOIN cmsdesynpuf23m.concept ON concept_id_2=concept_id) JOIN cmsdesynpuf23m.concept_ancestor ON concept_id=ancestor_concept_id )  JOIN cmsdesynpuf23m.drug_exposure dr1 ON concept_id=drug_concept_id) ON pe1.person_id=dr1.person_id);"

In [12]:
results = tool.execute_sql_query(executable_sql)
results

,count
0,610409


-----